<a href="https://colab.research.google.com/github/tsato-code/colab_notebooks/blob/master/emotion_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 参考資料
[Python で日本語文章の感情分析を簡単に試す (with google colab)](https://jupyterbook.hnishi.com/language-models/easy_try_sentiment_analysis.html)

In [1]:
%%capture capt

# MeCabのインストール
!apt install mecab libmecab-dev mecab-ipadic-utf8
!pip install mecab-python3

# mecab-ipadic-NEologdのインストール
!apt install git make curl xz-utils file
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git
!echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n -a

# Ref: https://qiita.com/Fulltea/items/90f6ebe6dcceaf64eaef
# Ref: https://qiita.com/SUZUKI_Masaya/items/685000d569452585210c

!ln -s /etc/mecabrc /usr/local/etc/mecabrc
# Ref: https://qiita.com/Naritoshi/items/8f55d7d5cce9ce414395

In [2]:
# 感情分析のためのライブラリ
!pip install -q asari oseti pymlask
# asari==0.0.4 requires Janome==0.3.7
# see https://github.com/Hironsan/asari/issues/9#issuecomment-695706645
!pip install Janome==0.3.7

     |████████████████████████████████| 10.4MB 3.9MB/s 
     |████████████████████████████████| 81kB 8.8MB/s 
     |████████████████████████████████| 71kB 7.6MB/s 
     |████████████████████████████████| 19.7MB 1.4MB/s 
     |████████████████████████████████| 61kB 7.7MB/s 
     |████████████████████████████████| 133kB 43.1MB/s 
     |████████████████████████████████| 20.7MB 53.1MB/s 
  Found existing installation: Janome 0.4.1
    Uninstalling Janome-0.4.1:
      Successfully uninstalled Janome-0.4.1


In [3]:
list_text = [
             'この人は、この世の中で、いちばんしあわせな人にちがいありません。',
             '芝居小屋もすばらしいし、お客さんもすばらしい人たちでした。',
             'もし中世の時代だったら、おそらく、火あぶりにされたでしょうよ。',
             'みんなのうるさいことといったら、まるで、ハエがびんの中で、ブンブンいっているようでした。',
             'われわれ人間が、こういうことを考えだすことができるとすれば、われわれは、地の中にうめられるまでに、もっと長生きできてもいいはずだが'
]

## asari

In [4]:
%%capture capt

# シンプルな動作確認
from asari.api import Sonar
sonar = Sonar()
res = sonar.ping(text="広告多すぎる♡")
res 

In [5]:
list(map(sonar.ping, list_text))

[{'classes': [{'class_name': 'negative', 'confidence': 0.10382535749585702},
   {'class_name': 'positive', 'confidence': 0.896174642504143}],
  'text': 'この人は、この世の中で、いちばんしあわせな人にちがいありません。',
  'top_class': 'positive'},
 {'classes': [{'class_name': 'negative', 'confidence': 0.035517582235360945},
   {'class_name': 'positive', 'confidence': 0.964482417764639}],
  'text': '芝居小屋もすばらしいし、お客さんもすばらしい人たちでした。',
  'top_class': 'positive'},
 {'classes': [{'class_name': 'negative', 'confidence': 0.5815274190768989},
   {'class_name': 'positive', 'confidence': 0.41847258092310113}],
  'text': 'もし中世の時代だったら、おそらく、火あぶりにされたでしょうよ。',
  'top_class': 'negative'},
 {'classes': [{'class_name': 'negative', 'confidence': 0.2692695045573754},
   {'class_name': 'positive', 'confidence': 0.7307304954426246}],
  'text': 'みんなのうるさいことといったら、まるで、ハエがびんの中で、ブンブンいっているようでした。',
  'top_class': 'positive'},
 {'classes': [{'class_name': 'negative', 'confidence': 0.050528495655525495},
   {'class_name': 'positive', 'confidence': 0.94

## oseti

In [6]:
# シンプルな動作確認
import oseti

analyzer = oseti.Analyzer()
analyzer.analyze('天国で待ってる。')

[1.0]

In [7]:
list(map(analyzer.analyze, list_text))

[[0.0], [1.0], [0], [0], [1.0]]

## pymlask

In [8]:
# シンプルな動作確認
import mlask
emotion_analyzer = mlask.MLAsk()
emotion_analyzer.analyze('彼のことは嫌いではない！(;´Д`)')
# => {'text': '彼のことは嫌いではない！(;´Д`)',
#     'emotion': defaultdict(<class 'list'>,{'yorokobi': ['嫌い*CVS'], 'suki': ['嫌い*CVS']}),
#     'orientation': 'POSITIVE',
#     'activation': 'NEUTRAL',
#     'emoticon': ['(;´Д`)'],
#     'intension': 2,
#     'intensifier': {'exclamation': ['！'], 'emotikony': ['´Д`', 'Д`', '´Д', '(;´Д`)']},
#     'representative': ('yorokobi', ['嫌い*CVS'])
#     }

{'activation': 'NEUTRAL',
 'emoticon': ['(;´Д`)'],
 'emotion': defaultdict(list, {'suki': ['嫌い*CVS'], 'yorokobi': ['嫌い*CVS']}),
 'intensifier': {'emotikony': ['´Д`', 'Д`', '´Д', '(;´Д`)'],
  'exclamation': ['！']},
 'intension': 2,
 'orientation': 'POSITIVE',
 'representative': ('yorokobi', ['嫌い*CVS']),
 'text': '彼のことは嫌いではない！(;´Д`)'}

In [9]:
# せっかくなので、neologd 辞書を使ってみる

# mecab-ipadic-neologd のインストール先を調べる
import subprocess

cmd='echo `mecab-config --dicdir`"/mecab-ipadic-neologd"'
path = (subprocess.Popen(cmd, stdout=subprocess.PIPE,
                           shell=True).communicate()[0]).decode('utf-8')
                           
emotion_analyzer = mlask.MLAsk('-d {0}'.format(path))  # Use other dictionary

list(map(emotion_analyzer.analyze, list_text))

[{'activation': 'NEUTRAL',
  'emoticon': None,
  'emotion': defaultdict(list, {'yorokobi': ['しあわせ']}),
  'intensifier': {},
  'intension': 0,
  'orientation': 'POSITIVE',
  'representative': ('yorokobi', ['しあわせ']),
  'text': 'この人は、この世の中で、いちばんしあわせな人にちがいありません。'},
 {'emotion': None, 'text': '芝居小屋もすばらしいし、お客さんもすばらしい人たちでした。'},
 {'emotion': None, 'text': 'もし中世の時代だったら、おそらく、火あぶりにされたでしょうよ。'},
 {'emotion': None, 'text': 'みんなのうるさいことといったら、まるで、ハエがびんの中で、ブンブンいっているようでした。'},
 {'emotion': None,
  'text': 'われわれ人間が、こういうことを考えだすことができるとすれば、われわれは、地の中にうめられるまでに、もっと長生きできてもいいはずだが'}]

## huggingface の bert-base-japanese-sentiment

In [10]:
# 必要なライブラリのインストール
!pip install -q transformers

     |████████████████████████████████| 2.3MB 4.0MB/s 
     |████████████████████████████████| 3.3MB 17.2MB/s 
     |████████████████████████████████| 901kB 42.1MB/s 


In [11]:
%%capture capt

from transformers import BertTokenizer, BertForSequenceClassification, pipeline

tokenizer = BertTokenizer.from_pretrained("daigo/bert-base-japanese-sentiment")
model = BertForSequenceClassification.from_pretrained("daigo/bert-base-japanese-sentiment")

In [12]:
# シンプルな動作確認

print(pipeline("sentiment-analysis",model=model, tokenizer=tokenizer)("私は幸福である。"))

[{'label': 'ポジティブ', 'score': 0.9934489130973816}]


In [13]:
sentiment_analyzer = pipeline("sentiment-analysis",model=model, tokenizer=tokenizer)

list(map(sentiment_analyzer, list_text))

[[{'label': 'ポジティブ', 'score': 0.9596116542816162}],
 [{'label': 'ポジティブ', 'score': 0.6044701933860779}],
 [{'label': 'ポジティブ', 'score': 0.8851077556610107}],
 [{'label': 'ポジティブ', 'score': 0.6943467855453491}],
 [{'label': 'ポジティブ', 'score': 0.5758240222930908}]]

In [14]:
# 上記すべてポジティヴ判定になってしまったので、他の例でも試してみる

list(map(sentiment_analyzer, ['最悪だ', '今日は暑い', 'こんにちは', 'ふつう']))

[[{'label': 'ネガティブ', 'score': 0.9891454577445984}],
 [{'label': 'ネガティブ', 'score': 0.8046908974647522}],
 [{'label': 'ポジティブ', 'score': 0.9862995147705078}],
 [{'label': 'ポジティブ', 'score': 0.9928306937217712}]]